In [3]:
import pandas as pd
import os
import boto3
import io # Necessário para ler o conteúdo do S3 como arquivo

# --- 1. Configurações de Acesso ao S3 ---
# As variáveis de ambiente (chaves AWS, bucket, região) são lidas do Docker Compose
bucket_name = os.environ.get("S3_BUCKET_NAME")
region = os.environ.get("AWS_REGION")
s3_prefix = "raw_data/"

s3_client = boto3.client(
    's3',
    aws_access_key_id=os.environ.get("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.environ.get("AWS_SECRET_ACCESS_KEY"),
    region_name=region
)

# --- 2. Encontrar o Arquivo Mais Recente ---
try:
    # Lista todos os objetos (arquivos) na pasta raw_data/
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=s3_prefix)
    
    if 'Contents' not in response:
        print("Erro: Nenhuma chave 'Contents' encontrada. O bucket pode estar vazio ou a permissão está errada.")
        # Se for um erro de permissão, o problema está nas chaves IAM no seu .env
        exit()

    # Encontra o arquivo mais recentemente modificado (seu dataset)
    latest_file = max(response['Contents'], key=lambda x: x['LastModified'])
    s3_key_full = latest_file['Key']
    
    print(f"Arquivo encontrado no S3: s3://{bucket_name}/{s3_key_full}")

    # --- 3. Baixar e Ler o Conteúdo do Arquivo ---
    # Obtém o objeto (o arquivo) do S3
    obj = s3_client.get_object(Bucket=bucket_name, Key=s3_key_full)
    
    # Lê o conteúdo binário do objeto
    file_content = obj['Body'].read()
    
    # Usa io.BytesIO para tratar o conteúdo binário como um arquivo na memória
    df = pd.read_csv(io.BytesIO(file_content))
    
    print("\nDataFrame lido com sucesso no Pandas!")

except Exception as e:
    print(f"Erro Crítico ao acessar o S3 ou ler o arquivo. Verifique as credenciais. Erro: {e}")
    exit()

# --- 4. Exploração Inicial (EDA) ---
print("\n--- Informações do DataFrame (df.info()) ---")
df.info()

print("\n--- Primeiras 5 Linhas (df.head()) ---")
print(df.head())

print(f"\nO DataFrame tem {df.shape[0]} linhas e {df.shape[1]} colunas.")

Arquivo encontrado no S3: s3://ml-bucket-heart/raw_data/heart_statlog_cleveland_hungary_final_1764200065.csv

DataFrame lido com sucesso no Pandas!

--- Informações do DataFrame (df.info()) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1190 entries, 0 to 1189
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  1190 non-null   int64  
 1   sex                  1190 non-null   int64  
 2   chest pain type      1190 non-null   int64  
 3   resting bp s         1190 non-null   int64  
 4   cholesterol          1190 non-null   int64  
 5   fasting blood sugar  1190 non-null   int64  
 6   resting ecg          1190 non-null   int64  
 7   max heart rate       1190 non-null   int64  
 8   exercise angina      1190 non-null   int64  
 9   oldpeak              1190 non-null   float64
 10  ST slope             1190 non-null   int64  
 11  target               1190 non-null   int64  